In [ ]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import time

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.get('https://twitter.com/AlexReibman/status/1767296114267754626')

In [ ]:
page_content = driver.page_source

In [ ]:
comments = []

In [ ]:
def scrape_comments(page_content):
    soup = BeautifulSoup(page_content, 'html.parser')
    comments_raw = []
    for x in soup.find_all('div', class_='css-175oi2r r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu'):
        if x.find('div', class_='css-175oi2r r-1awozwy r-18u37iz r-1cmwbt1 r-1wtj0ep').text == 'Ad':
            continue
        else:
            comments_raw.append(x)
    
    comments = []
    
    for comment in comments_raw:
        
        comment_data = {}
        
        display_element = comment.find('span', class_='css-1qaijid r-dnmrzs r-1udh08x r-3s2u2q r-bcqeeo r-qvutc0 r-poiln3')
        if display_element:
            comment_data['Display Name'] = display_element.text
        else:
            'N/A'
            
        user_element = comment.find('div', class_='css-1rynq56 r-dnmrzs r-1udh08x r-3s2u2q r-bcqeeo r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-18u37iz r-1wvb978')
        if user_element:
            comment_data['Username'] = user_element.text
        else:
            'N/A'
            
        url_element = comment.find('a', class_='css-1rynq56 r-bcqeeo r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-xoduu5 r-1q142lx r-1w6e6rj r-9aw3ui r-3s2u2q r-1loqt21')
        if url_element:
            comment_data['URL'] = 'https://www.twitter.com' + url_element['href']
        else:
            'N/A'
            
        time_element = comment.find('time')
        if time_element:
            comment_data['Timestamp'] = time_element.text
        else:
            'N/A'
            
        text_element = comment.find('div', class_='css-1rynq56 r-8akbws r-krxsd3 r-dnmrzs r-1udh08x r-bcqeeo r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-bnwqim')
        if text_element:
            comment_data['Text'] = text_element.text
        else:
            'N/A'
            
        like_element = comment.find(attrs={'data-testid': 'like'})
        if like_element:
            comment_data['No. of Likes'] = like_element.text
        else:
            'N/A'
            
        retweet_element = comment.find(attrs={'data-testid': 'retweet'})
        if retweet_element:
            comment_data['No. of Retweets'] = retweet_element.text
        else:
            'N/A'
            
        comments.append(comment_data)
        
    df = pd.DataFrame(comments)
    
    return df

In [ ]:
df_temp = scrape_comments(page_content)

In [ ]:
df = pd.concat([df, df_temp], ignore_index=True)

In [ ]:
out_path = 'C:\\Users\\BINARY COMPUTERS\\Documents\\Twitter.xlsx'
writer = pd.ExcelWriter(out_path, engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [ ]:
# Clean up
driver.quit()